In [1]:
from collections import defaultdict
from Bio.SeqIO.FastaIO import SimpleFastaParser as sfp
from Bio import SeqIO, SearchIO, Entrez
import seaborn as sns
import pandas as pd
import os, glob, math, re, gzip, skbio, time
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import warnings
from Bio import Entrez
warnings.filterwarnings('ignore')
Entrez.email = "ajaffe@stanford.edu"

In [2]:
import sys
sys.path.insert(0, '/home/users/ajaffe/notebooks/modules/')
import parse

In [3]:
def cmdir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def scaffold(gene):
    if gene != "None":
        try: return re.search("(.+?)_[0-9]+$", gene).group(1)
        except: print(gene)

def sbatch(name, cpus, cmd):
    return "sbatch -J %s -p serc -t 1- -c %d --mem %dG --wrap '%s'" %(name, cpus, cpus*8, cmd)

In [4]:
rootdir = "/scratch/users/ajaffe/photoeco/"
cmdir(rootdir + "context")

# gather data

In [5]:
cmdir(rootdir + "context/external")
cmdir(rootdir + "context/internal")

### cyano

In [6]:
cyano = pd.read_csv(rootdir + "rubisco_table.tsv", sep="\t")
cyano.head()

,gene,hmm,scaffold,genome_name,lineage,protein_sequence
0,OceanDNA-b16234_00052_2,II,OceanDNA-b16234_00052,OceanDNA-b16234,AMZ IB,MALNQSSRYADLNLDEQTLINEGRHILCAYRMKPKSGTGYLEGAAH...
1,ERR868457.100.1_contig_163_1,II,ERR868457.100.1_contig_163,ERR868457.100.1,AMZ IB,MALNQSSRYADLNLDEQTLINEGRHILCAYRMKPKSGTGYLEGAAH...
2,JASLWS010000001.1_45,II,JASLWS010000001.1,GCA_030740755.1_ASM3074075v1_genomic,AMZ IB,MALNQSSRYADLNLDEQTLINEGRHILCAYRMKPKSGTGYLEGAAH...
3,JASLTS010000061.1_1,II,JASLTS010000061.1,GCA_030742835.1_ASM3074283v1_genomic,AMZ IB,MALNQSSRYADLNLDEQTLINEGRHILCAYRMKPKSGTGYLEGAAH...
4,Ga0078703_1006_31,II,Ga0078703_1006,2626541517,AMZ IB,MALNQSSRYADLNLDEQTLINEGRHILCAYRMKPKSGTGYLEGAAH...


In [7]:
for key, row in cyano.query("hmm=='II'").iterrows():
    
    faa = glob.glob(rootdir + "genomes/filtered/" + row["genome_name"] + ".genes.faa")
    out = rootdir + "context/internal/" + row["scaffold"] + ".faa"
    
    with open(out, "w") as outfile:
        for record in sfp(open(faa[0])):
            orf = record[0].split(" # ")[0]
            scaf = "_".join(orf.split("_")[:-1])
            if scaf == row["scaffold"]:
                outfile.write(">%s\n%s\n" %(orf, record[1]))

### related

In [7]:
rubiscos = pd.read_csv(rootdir + "all_rubiscos.tsv", sep="\t")
neighbors = [line.replace(" ", "_").strip() for line in \
             open(rootdir + "context/neighbors.txt").readlines()]
genes = rubiscos[(rubiscos["cluster_name"].isin(neighbors)) & 
                 (~rubiscos["gene"].isin(cyano["gene"]))]["gene"].to_list()
genes[5:10]

['WP_092620206.1',
 'TARA_110.SAMEA2622429.380.0.22-3_1648520_4',
 'TARA_056.SAMEA2620666.1000.0.22-3_9262_5',
 'TARA_137.SAMEA2623295.40.0.22-3_632769_33',
 'WP_043107373.1']

In [8]:
extinfo = defaultdict(list)

for gene in genes:
        
    try:
        
        handle = Entrez.efetch(db="protein", id=gene, retmode="xml")
        soup = BeautifulSoup(handle, "html.parser")
        defn = soup.find("gbseq_definition").string
        handle.close()

        if "MAG" in defn:

            source = soup.find("gbseq_source-db").string.split(" ")[1]
            handle = Entrez.efetch(db="nucleotide", id=source, rettype="gb", retmode="text")

            for record in SeqIO.parse(handle, "genbank"):

                name = record.name
                outfile = rootdir + "context/external/" + record.name + ".faa"

                with open(outfile, "w") as out:
                    for feature in record.features:
                        if feature.type == "CDS":
                            if "translation" in feature.qualifiers:
                                out.write(">%s\n%s\n" %(feature.qualifiers["protein_id"][0], 
                                                        feature.qualifiers["translation"][0]))
                        if feature.type == "source":
                            extinfo["rubisco"].append(gene)
                            extinfo["filename"].append(name)
                            for field in ["organism", "isolate", "isolation_source", "metagenome_source"]:
                                try:
                                    extinfo[field].append(feature.qualifiers[field][0])
                                except: extinfo[field].append("None")
            handle.close()
        
        else: print("%s found but not in a MAG." %(gene))
    
    except Exception as error: print("%s failed on %s error" %(gene, type(error)))
        
    time.sleep(1)

extdf = pd.DataFrame(extinfo)

II.CBBH failed on <class 'urllib.error.HTTPError'> error
RBC2.24 failed on <class 'urllib.error.HTTPError'> error
RBC4.70 failed on <class 'urllib.error.HTTPError'> error
WP_043107373.1 found but not in a MAG.
WP_045226995.1 found but not in a MAG.
WP_092620206.1 found but not in a MAG.
TARA_110.SAMEA2622429.380.0.22-3_1648520_4 failed on <class 'urllib.error.HTTPError'> error
TARA_056.SAMEA2620666.1000.0.22-3_9262_5 failed on <class 'urllib.error.HTTPError'> error
TARA_137.SAMEA2623295.40.0.22-3_632769_33 failed on <class 'urllib.error.HTTPError'> error
WP_043107373.1 found but not in a MAG.
WP_045226995.1 found but not in a MAG.
WP_092620206.1 found but not in a MAG.
WP_205430433.1 found but not in a MAG.
WP_153249906.1 found but not in a MAG.


# annotation

### functional

In [13]:
# combine
with open(rootdir + "context/neighbors.faa", "w") as out:
    for scaf in glob.glob(rootdir + "context/*ternal/*"):
        for record in sfp(open(scaf)):
            out.write(">%s\n%s\n" %(record[0], record[1]))

In [14]:
#kofam
kocall = "exec_annotation -o %s %s -p %s -k %s --cpu 20 -f detail" \
    %(rootdir + "context/neighbors.kfscan.txt",
      rootdir + "context/neighbors.faa",
     "/home/groups/dekas/software/kofamscan/profiles/prokaryote.hal",
      "/home/groups/dekas/software/kofamscan/ko_list")
print(sbatch("kfscan", 20, kocall))

sbatch -J kfscan -p serc -t 1- -c 20 --mem 160G --wrap 'exec_annotation -o /scratch/users/ajaffe/photoeco/context/neighbors.kfscan.txt /scratch/users/ajaffe/photoeco/context/neighbors.faa -p /home/groups/dekas/software/kofamscan/profiles/prokaryote.hal -k /home/groups/dekas/software/kofamscan/ko_list --cpu 20 -f detail'


In [9]:
kresults = parse.kofamscan(rootdir + "context/neighbors.kfscan.txt", 1e-20)
kresults.head(2)

,gene,ko,threshold,score,eval,def
24622,MBP81482.1,K18989,990.27,1420.1,0.0,multidrug efflux pump
19374,MBT3263320.1,K18989,990.27,1408.1,0.0,multidrug efflux pump


In [17]:
#pfam
cmd = "hmmsearch --cpu 20 --domtblout %s /oak/stanford/groups/dekas/db/Pfam-A.hmm %s" %(rootdir + "context/neighbors.pfam.txt",
                                                                               rootdir + "context/neighbors.faa")
print(sbatch("pfam", 20, cmd))

sbatch -J pfam -p serc -t 1- -c 20 --mem 160G --wrap 'hmmsearch --cpu 20 --domtblout /scratch/users/ajaffe/photoeco/context/neighbors.pfam.txt /oak/stanford/groups/dekas/db/Pfam-A.hmm /scratch/users/ajaffe/photoeco/context/neighbors.faa'


In [10]:
pfam = parse.domtable(rootdir + "context/neighbors.pfam.txt")
pfam = pfam.query("eval<1e-20").query("hmm_overall_cov>0.50")

pfam_info = defaultdict(list)

for gene in pfam["gene"].unique():
    pfams = pfam[pfam["gene"]==gene]["hmm"].to_list()
    evals = pfam[pfam["gene"]==gene]["eval"].to_list()
    pfam_info["gene"].append(gene)
    pfam_info["pfams"].append(" + ".join(pfams))
    pfam_info["highest_eval"].append(max(evals))

pfam_df = pd.DataFrame(pfam_info)
pfam_df.head(2)

,gene,pfams,highest_eval
0,MBT3263309.1,2-Hacid_dh + 2-Hacid_dh_C,7.500000e-27
1,MBP81497.1,2-Hacid_dh + 2-Hacid_dh_C,5.300000e-26


### protein family

In [27]:
cmdir(rootdir + "context/clustering")
cmdir(rootdir + "context/clustering/fasta")

In [15]:
# run mmseqs
makedb = "mmseqs createdb %s neighbors.db" %(rootdir + "context/neighbors.faa")
cluster = "mmseqs cluster --cov-mode 0 --threads 4 neighbors.db neighbors.cluster tmp"
process = "mmseqs createtsv neighbors.db neighbors.db neighbors.cluster neighbors.cluster.tsv"
print(process)

mmseqs createtsv neighbors.db neighbors.db neighbors.cluster neighbors.cluster.tsv


In [19]:
# output orf2bin and fastas
results = pd.read_csv(rootdir + "context/neighbors.cluster.tsv", sep="\t", header=None)
tmp = {record[0]:record[1] for record in sfp(open(rootdir + "context/neighbors.faa"))}
fams = {}

for i, centroid in enumerate(results[0].unique()):
    
    with open(rootdir + "context/clustering/fasta/family%d.fasta" %(i), "w") as out:
        
        subtable = results[results[0]==centroid]
        for key, row in subtable.iterrows():
            fams[row[1]] = "family" + str(i)
            out.write(">%s\n%s\n" %(row[1], tmp[row[1]]))
            
tmp = {}

### taxonomic

In [20]:
from scipy import stats

In [22]:
dmnd = "diamond blastp -d /$OAK/db/uniref100.dmnd -q %s -o %s --threads 20 -b8 -c1" \
            %(rootdir + "context/neighbors.faa",
              rootdir + "context/neighbors.dmnd.results")
print(sbatch("diamond", 20, dmnd))

sbatch -J diamond -p serc -t 1- -c 20 --mem 160G --wrap 'diamond blastp -d /$OAK/db/uniref100.dmnd -q /scratch/users/ajaffe/photoeco/context/neighbors.faa -o /scratch/users/ajaffe/photoeco/context/neighbors.dmnd.results --threads 20 -b8 -c1'


In [24]:
# concatenate + collect taxonomy
dmnd = skbio.io.read(rootdir + "context/neighbors.dmnd.results", 
                     format="blast+6", into=pd.DataFrame, default_columns=True)
# compute coverage
faalens = {record[0].split(" ")[0]: len(record[1]) for record \
           in sfp(open(rootdir + "context/neighbors.faa"))}
dmnd["qlen"] = dmnd["qseqid"].map(faalens)
dmnd["qcov"] = dmnd.apply(lambda x: (x["qend"]-x["qstart"])/x["qlen"], axis=1)
# choose best hits for each
dmnd = dmnd.sort_values(["bitscore", "qcov"], ascending=[False,False]).drop_duplicates("qseqid")
# filter for min cov /eval
dmnd = dmnd[(dmnd["evalue"]<1e-20) & (dmnd["qcov"]>0.70)]
dmnd.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qlen,qcov
16378,SRR12426458.100.119_scaffold_2_41,UniRef100_A0A7Y0KUK1,81.6,1573.0,288.0,1.0,1.0,1572.0,1.0,1573.0,0.0,2653.0,1578,0.995564
22968,MBR01787.1,UniRef100_A0A2E8MDQ0,100.0,994.0,0.0,0.0,1.0,994.0,1.0,994.0,0.0,1944.0,994,0.998994
25132,MBP81482.1,UniRef100_A0A2E8CKT0,100.0,1047.0,0.0,0.0,1.0,1047.0,1.0,1047.0,0.0,1938.0,1047,0.999045
21842,MBT3263320.1,UniRef100_A0A2E8CKT0,86.1,1046.0,145.0,0.0,1.0,1046.0,1.0,1046.0,0.0,1703.0,1048,0.997137
5339,Ga0078703_1006_1,UniRef100_UPI0007B3805C,82.0,972.0,174.0,1.0,1.0,971.0,1.0,972.0,0.0,1652.0,971,0.998970


In [25]:
# map to uniref ids
with open(rootdir + "context/uniref_ids.txt", "w") as out:
    for uid in dmnd["sseqid"].unique():
        out.write(uid.split("_")[1] + "\n")

In [39]:
uni = []
for item in glob.glob(rootdir + "context/idmapping*tsv"):
    table = pd.read_csv(item,sep="\t", dtype=str)
    table.columns = ["from", "entry", "organism", "organism_id"]
    uni.append(table)
unis = pd.concat(uni).fillna("None")
unis = unis.query("organism_id!='None'")
unis["taxid"] = unis["organism_id"].apply(lambda x: stats.mode(x.split("; "))[0][0])

In [40]:
with open(rootdir + "context/ncbi_tax.txt", "w") as out:
    for tax in unis["taxid"].unique():
        if tax != "None":
            out.write(tax + "\n")

In [41]:
lineage_info = defaultdict(list)

for xml in glob.glob(rootdir + "/context/*xml"):
    
    for block in BeautifulSoup(open(xml), "xml").findAll("Taxon"):
        
        lineage, phylum, classe = "None", "None", "None"

        if block.find("Lineage"):
            lineage = block.find("Lineage").string

        for level in block.findAll("Taxon"):
            if level.find("Rank").string=="phylum":
                phylum = level.find("ScientificName").string
            elif level.find("Rank").string=="class":
                classe = level.find("ScientificName").string
                
        if (phylum == classe == 'None') and (lineage!='None'):
            phylum = lineage.split("; ")[-1]

        lineage_info["taxid"].append(block.find("TaxId").string)
        lineage_info["lineage"].append(lineage)
        lineage_info["phylum"].append(";".join([phylum, classe]))

lineage_df = pd.DataFrame(lineage_info).query("lineage!='None'")
unis = unis.merge(lineage_df, how="left", on="taxid")
unis.head()

,from,entry,organism,organism_id,taxid,lineage,phylum
0,A0A7Y0KUK1,A0A7Y0KUK1,Prochlorococcus sp. P1361,2729589,2729589,cellular organisms; Bacteria; Terrabacteria gr...,Cyanobacteriota;Cyanophyceae
1,A0A2E8MDQ0,A0A2E8MDQ0,Acidiferrobacteraceae bacterium,2024893,2024893,cellular organisms; Bacteria; Pseudomonadota; ...,Pseudomonadota;Gammaproteobacteria
2,A0A2E8CKT0,A0A2E8CKT0,Acidiferrobacteraceae bacterium,2024893,2024893,cellular organisms; Bacteria; Pseudomonadota; ...,Pseudomonadota;Gammaproteobacteria
3,A0A7C1Y0M4,A0A7C1Y0M4,Gammaproteobacteria bacterium,1913989,1913989,cellular organisms; Bacteria; Pseudomonadota; ...,Pseudomonadota;Gammaproteobacteria
4,A0A2E8CKP1,A0A2E8CKP1,Acidiferrobacteraceae bacterium,2024893,2024893,cellular organisms; Bacteria; Pseudomonadota; ...,Pseudomonadota;Gammaproteobacteria


In [43]:
# add back in
dmnd["entry"] = dmnd["sseqid"].apply(lambda x: x.split("_")[1])
dmnd = dmnd.merge(unis[["entry", "phylum"]], how="left").fillna("None")
dmnd.tail()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qlen,qcov,entry,phylum
1064,SRR25584942.100.22_scaffold_0_33,UniRef100_A0A560LLE7,100.0,51.0,0.0,0.0,1.0,51.0,1.0,51.0,3.890000e-22,89.4,52,0.961538,A0A560LLE7,None
1065,SRR25584950.100.58_scaffold_5_18,UniRef100_A0A560LLE7,100.0,51.0,0.0,0.0,1.0,51.0,1.0,51.0,3.890000e-22,89.4,52,0.961538,A0A560LLE7,None
1066,SRR11923207.100.13_scaffold_3_36,UniRef100_A0A560LLE7,100.0,51.0,0.0,0.0,1.0,51.0,1.0,51.0,3.890000e-22,89.4,52,0.961538,A0A560LLE7,None
1067,SRR11923208.100.54_scaffold_0_43,UniRef100_A0A560LLE7,100.0,51.0,0.0,0.0,1.0,51.0,1.0,51.0,3.890000e-22,89.4,52,0.961538,A0A560LLE7,None
1068,JASLWS010000001.1_18,UniRef100_A0A560LLE7,100.0,51.0,0.0,0.0,1.0,51.0,1.0,51.0,3.890000e-22,89.4,52,0.961538,A0A560LLE7,None


### combine

In [45]:
base = defaultdict(list)
tmp = {record[0]:record[1] for record in sfp(open(rootdir + "context/neighbors.faa"))}

for scaf in glob.glob(rootdir + "context/*ternal/*"):
    name = os.path.basename(scaf).replace(".faa","")
    for record in sfp(open(scaf)):
        base["scaffold"].append(name)
        base["gene"].append(record[0])

basedf = pd.DataFrame(base)
basedf = basedf.merge(kresults[["gene", "def"]], how="left").fillna("None")
basedf = basedf.merge(pfam_df[["gene", "pfams"]], how="left").fillna("None")
basedf["family"] = basedf["gene"].map(fams)
basedf = basedf.merge(dmnd[["qseqid", "phylum"]], how="left", left_on="gene", 
                      right_on="qseqid").fillna("None").drop("qseqid", axis=1)
basedf.columns = ["scaffold", "gene", "ko_def", "pfams", "family", "taxonomy"]
basedf["sequence"] = basedf["gene"].map(tmp)
basedf.drop_duplicates("gene").to_csv(rootdir + "genomic_context.csv", sep=",", index=False)
basedf.head(2)

,scaffold,gene,ko_def,pfams,family,taxonomy,sequence
0,ERR598946.100.35_scaffold_13,ERR598946.100.35_scaffold_13_1,None,None,family0,Cyanobacteriota;Cyanophyceae,MAAIRLIPAVLALSLTLVSCQSSSEKAANDQVKVAQGVELVCSARD...
1,ERR598946.100.35_scaffold_13,ERR598946.100.35_scaffold_13_2,GTP-binding protein,BipA_C + EFG_C + GTP_EFTU,family24,Cyanobacteriota;Cyanophyceae,MSSGGAEGKLDLTLRYPFATTPVGVSFPLVRLALMSAQAQAIRNIA...


### export + visualize

In [51]:
subset = basedf.query("scaffold=='SRR11923207.100.13_scaffold_3'").drop_duplicates("gene")

# recover prodigal info
info = defaultdict(list)

for record in sfp(open(rootdir + "genomes/filtered/SRR11923207.100.13.genes.faa")):
    info["gene"].append(record[0].split(" # ")[0])
    info["start"].append(int(record[0].split(" # ")[1]))
    info["end"].append(int(record[0].split(" # ")[2]))
    info["strand"].append(int(record[0].split(" # ")[3]))

infodf = pd.DataFrame(info)
infodf.head(2)

,gene,start,end,strand
0,SRR11923207.100.13_scaffold_0_1,524,1591,-1
1,SRR11923207.100.13_scaffold_0_2,1616,3277,-1


In [52]:
subset = subset.merge(infodf, how="left", on="gene")
# split into two
#subset["scaffold_mod"] = subset.apply(lambda x: x["scaffold"] + "A" if x["end"]<=27666 else x["scaffold"] + "B", axis=1)
subset[["scaffold", "start", "end", "strand", 
        "ko_def", "pfams", "family", "taxonomy"]].to_csv(rootdir + "genome_context_subset.csv", index=False)